## Bribes research

In [43]:
import math

import pandas as pd
import requests
from IPython.core.display_functions import display

from notebooks.bribes_research.helpers import fetch_token_prices

BADGER_BRIBE = 40000

BRIBES_URL = "https://api.llama.airforce/bribes"
DASHBOARD_URL = "https://api.llama.airforce//dashboard"
TABLE_COLUMNS = ["Protocol", "BADGER total bribe in $", "Bribes Portion %", "Bribes Portion", "Dilution", "$I/O",]
PROTOCOL_CVX = "CVX"
PROTOCOL_AURA = "AURA"

table_rows = []
token_prices = fetch_token_prices(['badger-dao'])
badger_price = token_prices['badger-dao']['usd']
badger_bribe_in_dollar = BADGER_BRIBE * badger_price

cvx_bribes = requests.post(BRIBES_URL, json={'platform': "votium", 'protocol': "cvx-crv"}).json()
cvx_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-votium"}).json()
aura_bribes = requests.post(BRIBES_URL, json={'platform': "hh", 'protocol': "aura-bal"}).json()
aura_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-aura"}).json()

total_bribes_cvx = sum([bribe['amountDollars'] for bribe in cvx_bribes['epoch']['bribes']])
dollar_spent_on_bribes_cvx = cvx_dash['dashboard']['rewardPerDollarBribe']

total_bribes_aura = sum([bribe['amountDollars'] for bribe in aura_bribes['epoch']['bribes']])
dollar_spent_on_bribes_aura = aura_dash['dashboard']['rewardPerDollarBribe']

cvx_dilution_100 = badger_bribe_in_dollar / total_bribes_cvx * 100
aura_dilution_100 = badger_bribe_in_dollar / total_bribes_aura * 100

cvx_in_out_100 = (100 - cvx_dilution_100) * dollar_spent_on_bribes_cvx
table_rows.append(
    [PROTOCOL_CVX, badger_bribe_in_dollar, (badger_bribe_in_dollar / badger_bribe_in_dollar) * 100, badger_bribe_in_dollar, cvx_dilution_100, cvx_in_out_100]
)
aura_in_out_100 = (100 - aura_dilution_100) * dollar_spent_on_bribes_aura
table_rows.append(
    [PROTOCOL_AURA, badger_bribe_in_dollar, (badger_bribe_in_dollar / badger_bribe_in_dollar) * 100, badger_bribe_in_dollar, aura_dilution_100, aura_in_out_100]
)

# Rebalancing bribe values
cvx_dilution_50 = ((badger_bribe_in_dollar / 2) / total_bribes_cvx * 100)
aura_dilution_50 = ((badger_bribe_in_dollar / 2) / total_bribes_aura * 100)

cvx_in_out_50 = (100 - cvx_dilution_50) * dollar_spent_on_bribes_cvx
table_rows.append(
    [PROTOCOL_CVX, badger_bribe_in_dollar, ((badger_bribe_in_dollar / 2) / badger_bribe_in_dollar) * 100, badger_bribe_in_dollar / 2, cvx_dilution_50, cvx_in_out_50]
)
aura_in_out_50 = (100 - aura_dilution_50) * dollar_spent_on_bribes_aura
table_rows.append(
    [PROTOCOL_AURA, badger_bribe_in_dollar, ((badger_bribe_in_dollar / 2) / badger_bribe_in_dollar) * 100, badger_bribe_in_dollar / 2, aura_dilution_50, aura_in_out_50]
)

# Rebalance here
cvx_in_out_floating = cvx_in_out_50
aura_in_out_floating = aura_in_out_50
rebalancing_step = 0.01  # in %
cvx_bribe_portion = aura_bribe_portion = badger_bribe_in_dollar / 2
while not math.isclose(cvx_in_out_floating, aura_in_out_floating, abs_tol=0.5):
    if cvx_in_out_floating > aura_in_out_floating:
        # Take bribe portion from aura and give it to cvx
        bribe_stolen = aura_bribe_portion * rebalancing_step
        aura_bribe_portion -= bribe_stolen
        cvx_bribe_portion += bribe_stolen
    elif aura_in_out_floating > cvx_in_out_floating:
        # Take bribe portion from cvx and give it to aura
        bribe_stolen = aura_bribe_portion * rebalancing_step
        cvx_bribe_portion -= bribe_stolen
        aura_bribe_portion += bribe_stolen
    cvx_dilution =  cvx_bribe_portion / total_bribes_cvx * 100
    aura_dilution = aura_bribe_portion / total_bribes_aura * 100
    cvx_in_out_floating = (100 - cvx_dilution) * dollar_spent_on_bribes_cvx
    aura_in_out_floating = (100 - aura_dilution) * dollar_spent_on_bribes_aura

table_rows.append(
    [PROTOCOL_CVX, badger_bribe_in_dollar, round(cvx_bribe_portion / badger_bribe_in_dollar * 100, 2), cvx_bribe_portion, cvx_dilution, cvx_in_out_floating]
)
table_rows.append(
    [PROTOCOL_AURA, badger_bribe_in_dollar, round(aura_bribe_portion / badger_bribe_in_dollar * 100, 2), aura_bribe_portion, aura_dilution, aura_in_out_floating]
)
dataframe = pd.DataFrame(data=table_rows, columns=TABLE_COLUMNS)
display(dataframe)

,Protocol,BADGER total bribe in $,Bribes Portion %,Bribes Portion,Dilution,$I/O
0,CVX,171600.0,100.00,171600.000000,6.396820,137.757540
1,AURA,171600.0,100.00,171600.000000,55.507226,110.586181
2,CVX,171600.0,50.00,85800.000000,3.198410,142.464699
3,AURA,171600.0,50.00,85800.000000,27.753613,179.567406
4,CVX,171600.0,24.81,42578.046575,1.587203,144.835942
5,AURA,171600.0,75.19,129021.953425,41.734561,144.817951
